In [1]:
import numpy as np

In [2]:
class Processor:
    def __init__(self):
        self.processes = []
        self.gantt = []
        self.quant = 3
        self.status = []
    def add(self, proc, quant):
        self.processes = proc
        self.gantt = []
        self.quant = quant
        self.status = np.full(len(proc), False, dtype=bool)
        self.count = np.zeros((2, len(proc)))
        self.in_progress = np.full(len(proc), False, dtype=bool)
    def compute(self):
        iter = 0
        pointer = 0
        length = len(self.processes)
        stop = sum(self.processes)
        quant_left = self.quant
        flag = False
        while iter != stop:
            if quant_left == 0:
                quant_left = self.quant
                if not flag:
                    pointer += 1
                    if pointer == len:
                        pointer = 0
                flag = False
            while self.status[pointer]:
                if pointer == length - 1:
                    pointer = 0
                else:
                    pointer += 1
            self.processes[pointer] -= 1
            quant_left -= 1
            self.in_progress = np.full(length, False, dtype=bool)
            self.in_progress[pointer] = True
            self.gantt_add()
            if self.processes[pointer] == 0:
                flag = True
                self.status[pointer] = True
                if pointer == length - 1:
                    pointer = 0
                else:
                    pointer += 1
            iter += 1
        self.gantt = np.array(self.gantt)
        self.gantt = self.gantt.transpose()

    def compute1(self):
        priority = np.argsort(self.processes)
        iter = 0

        length = len(self.processes)
        stop = sum(self.processes)
        quant_left = self.quant
        flag = False
        iter_priority = 0
        pr_index = priority[iter_priority]
        pointer = pr_index
        while not np.all(self.status == True):
            if quant_left == 0:
                quant_left = self.quant
                if not flag:
                    pointer += 1
                    if pointer == len:
                        pointer = 0
                flag = False
            while self.status[pointer]:
                if pointer == length - 1:
                    pointer = 0
                else:
                    pointer += 1
            self.processes[pointer] -= 1
            quant_left -= 1
            self.in_progress = np.full(length, False, dtype=bool)
            self.in_progress[pointer] = True
            self.gantt_add()
            if self.processes[pointer] == 0:
                self.status[pointer] = True
                iter_priority += 1
                if iter_priority < length:
                    pr_index = priority[iter_priority]
                pointer = pr_index
            iter += 1
        self.gantt = np.array(self.gantt)
        self.gantt = self.gantt.transpose()

    def gantt_count(self):
        t1 = []
        t2 = []
        for i in range(self.gantt.shape[0]):
            t1.append(np.count_nonzero((self.gantt[i] == "P") + np.count_nonzero(self.gantt[i] == "0")))
        for i in range(self.gantt.shape[0]):
            t2.append(np.count_nonzero(self.gantt[i] == "0"))
        return t1, t2
    def gantt_add(self):
        temp = list(self.status * 1)
        index = np.where(self.in_progress == True)
        temp[index[0][0]] = "P"
        self.gantt.append(temp)

In [3]:
x = Processor()
x.add(np.array([5,6,8,4,1,2]), 3)
x.compute1()


In [4]:
x.gantt

array([['0', '0', '0', 'P', 'P', 'P', '0', '0', '0', '0', '0', '0', '0',
        '0', '0', 'P', 'P', '1', '1', '1', '1', '1', '1', '1', '1', '1'],
       ['0', '0', '0', '0', '0', '0', 'P', 'P', 'P', '0', '0', '0', '0',
        '0', '0', '0', '0', 'P', '0', '0', '0', '0', 'P', 'P', '1', '1'],
       ['0', '0', '0', '0', '0', '0', '0', '0', '0', 'P', 'P', 'P', '0',
        '0', '0', '0', '0', '0', 'P', 'P', 'P', '0', '0', '0', 'P', 'P'],
       ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', 'P',
        'P', 'P', '0', '0', '0', '0', '0', '0', 'P', '1', '1', '1', '1'],
       ['P', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
        '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1'],
       ['0', 'P', 'P', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
        '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']],
      dtype='<U11')

In [5]:
print("Общее время выполнения = ", sum(x.gantt_count()[0]),"Общее время ожидания = ", sum(x.gantt_count()[1]))
print("Среднее время выполнения = ", sum(x.gantt_count()[0])/6,"Среднее время ожидания = ", sum(x.gantt_count()[1])/6)

Общее время выполнения =  131 Общее время ожидания =  67
Среднее время выполнения =  21.833333333333332 Среднее время ожидания =  11.166666666666666
